Series
===

**Пример создания Series**

Одномерный набор данных. Отсутствующие данные записываются как np.nan. При вычислении среднего и других операций соответствующие функции не учитывают отсутствующие значения.


In [45]:
import numpy as np
import pandas as pd

l = [1, 3, 5, np.nan, 6, 8]
s = pd.Series(l)


In [46]:
print(s.values, type(s.values))

print(s.to_dict())

print(s.index)



[ 1.  3.  5. nan  6.  8.] <class 'numpy.ndarray'>
{0: 1.0, 1: 3.0, 2: 5.0, 3: nan, 4: 6.0, 5: 8.0}
RangeIndex(start=0, stop=6, step=1)


Основная информация о наборе данных: количество записей, среднее, стандартное отклонение, минимум, нижний квартиль, медиана, верхний квартиль, максимум, а так же тип данных. 


In [47]:
s.describe()


count    5.000000
mean     4.600000
std      2.701851
min      1.000000
25%      3.000000
50%      5.000000
75%      6.000000
max      8.000000
dtype: float64

**Индексация**


In [48]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s[2]

s[2] = 7
s

s[2:5]

s1 = s[1:]
s1

s2 = s[:-1]
s2


0    1.0
1    3.0
2    7.0
3    NaN
4    6.0
dtype: float64

В сумме s1+s2 складываются данные с одинаковыми индексами. Поскольку в s1 нет данного и индексом 0, а в s2 — с индексом 5, в s1+s2 в соответствующих позициях будет NaN.


In [49]:
s1 + s2


0     NaN
1     6.0
2    14.0
3     NaN
4    12.0
5     NaN
dtype: float64

К наборам данных можно применять функции из numpy.


In [50]:
np.exp(s1)
type(np.exp(s)), type(s)


(pandas.core.series.Series, pandas.core.series.Series)

При создании набора данных s мы не указали, что будет играть роль индекса. По умолчанию это последовательность неотрицательных целых чисел 0, 1, 2, ...


In [51]:
s.index


RangeIndex(start=0, stop=6, step=1)

Но можно создавать наборы данных с индексом, заданным списком.


In [52]:
i = list('abcdef')  # i == ['a', 'b', 'c', 'd', 'e', 'f']
s = pd.Series(l, index=i)
s

a    1.0
b    3.0
c    5.0
d    NaN
e    6.0
f    8.0
dtype: float64

Если индекс — строка, то вместо s['c'] можно писать s.c.


In [53]:
s['c']

5.0

In [54]:
s.c

5.0

Набор данных можно создать из словаря.


In [55]:
s = pd.Series({'a':1, 'b':2, 'c':0})
s

a    1
b    2
c    0
dtype: int64

Роль индекса может играть, скажем, последовательность дат или времени измерения и т.д.


In [56]:
d = pd.date_range('20160101', periods=6)
d


DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06'],
              dtype='datetime64[ns]', freq='D')

In [57]:
s = pd.Series(l, index=d)
s


2016-01-01    1.0
2016-01-02    3.0
2016-01-03    5.0
2016-01-04    NaN
2016-01-05    6.0
2016-01-06    8.0
Freq: D, dtype: float64

In [58]:
s[s > 3]


2016-01-03    5.0
2016-01-05    6.0
2016-01-06    8.0
dtype: float64

**Продвинутые операции с Series**


Если вы хотите посчитать разности соседних элементов, воспользуйтесь методом diff. Ключевое слово periods отвечает за то, с каким шагом будут считаться разности.


In [59]:
s.diff()


2016-01-01    NaN
2016-01-02    2.0
2016-01-03    2.0
2016-01-04    NaN
2016-01-05    NaN
2016-01-06    2.0
Freq: D, dtype: float64

Результат будет иметь тот же размер, но в начале появятся пропущенные значения. От них можно избавиться при помощи метода dropna.


In [60]:
s.diff().dropna()


2016-01-02    2.0
2016-01-03    2.0
2016-01-06    2.0
dtype: float64

Кумулятивные максимумы — от первого элемента до текущего. Первое значение кумулятивного максимума совпадает с первым значением исходного массива. Далее значение  𝑘 -го элемента есть максимум среди элементов до  𝑘 -го включительно. Аналогично минимум


In [61]:
s

2016-01-01    1.0
2016-01-02    3.0
2016-01-03    5.0
2016-01-04    NaN
2016-01-05    6.0
2016-01-06    8.0
Freq: D, dtype: float64

In [62]:
s.cummin()


2016-01-01    1.0
2016-01-02    1.0
2016-01-03    1.0
2016-01-04    NaN
2016-01-05    1.0
2016-01-06    1.0
Freq: D, dtype: float64

Кумулятивные суммы. Первое значение кумулятивной суммы совпадает с первым значением исходного массива. Далее значение  𝑘 -го элемента есть сумма элементов до  𝑘 -го включительно.


In [63]:
s.cumsum()


2016-01-01     1.0
2016-01-02     4.0
2016-01-03     9.0
2016-01-04     NaN
2016-01-05    15.0
2016-01-06    23.0
Freq: D, dtype: float64

Произвольные функции кумулятивным способом можно считать с помощью конструкции expanding. Например, так можно посчитать кумулятивные медианы. Будет не быстрее, чем вручную, но аккуратнее.


**Решение задач**
1. Что будет выведено в следующих случаях и почему


In [107]:
s = pd.Series([1, 2, 3, 4, 5, 6])
t = s[2:]+s[:2]
print(t)
print(t.dropna())


0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
dtype: float64
Series([], dtype: float64)


In [108]:
s[2:]

2    3
3    4
4    5
5    6
dtype: int64

In [109]:
s[:2]

0    1
1    2
dtype: int64

2. Посчитайте кумулятивное среднее квадратов разностей соседних элементов набора s.
От них можно избавиться при помощи метода dropna.


In [65]:
s.diff().dropna().expanding().apply(lambda x: np.mean(x**2))

1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
dtype: float64

DataFrames
--
Создание DataFrame.

DataFrame – двумерная таблица с данными. Имеет индекс и набор столбцов (возможно, имеющих разные типы). Таблицу можно построить, например, из словаря, значениями в котором являются одномерные наборы данных. Можно сказать, что это Series Series. При этом и каждый столбец, и каждая строка будут Series.


In [66]:
d = {'one': pd.Series(range(6), index=list('aaaefg'))}
df = pd.DataFrame(d)
df


,one
a,0
a,1
a,2
e,3
f,4
g,5


Для случая одного столбца не очень отличается от Series. Хотя визуализация другая, ибо датафрейм из одного столбца – это набор из одной Series, а не одна Series. Иногда для визуализации датафрейма в jupyter notebook / colab используют специальную инструкцию display: display(df).



In [67]:
display(df)

,one
a,0
a,1
a,2
e,3
f,4
g,5


Можно сбросить индекс! Это приведет к тому, что появится еще один столбец (который раньше был индексом) и создастся новый индекс по умолчанию.


In [68]:
df.reset_index()


,index,one
0,a,0
1,a,1
2,a,2
3,e,3
4,f,4
5,g,5


Создадим теперь датафрейм с несколькими столбцами. В идеальном случае длины столбцов должны быть одинаковыми! В нашем случае это не так, что в будущем приведет к неожиданным результатам. 


In [69]:
d = {'one': pd.Series(range(6), index=list('abdefg')),
     'two': pd.Series(range(7), index=list('abcdefg')),
     'three': pd.Series(np.random.normal(size=7), index=list('abcdefg'))}
df = pd.DataFrame(d)
df

,one,two,three
a,0.0,0,-0.146532
b,1.0,1,-0.770408
c,NaN,2,-0.470847
d,2.0,3,-0.982997
e,3.0,4,-1.061588
f,4.0,5,1.068391
g,5.0,6,0.238150


Несмотря на то, что в столбце one нет индекса c, таблица заполнена и на месте пропуска стоит np.nan, как и всегда. Даже если мы попросим вывести этот столбец, то в нем будут 7 элементов. Ибо при индексации мы обращаемся к датафрейму, а в нем строка c присутствует.


In [70]:
df['one']


a    0.0
b    1.0
c    NaN
d    2.0
e    3.0
f    4.0
g    5.0
Name: one, dtype: float64

In [71]:
df.index


Index(['a', 'b', 'c', 'd', 'e', 'f', 'g'], dtype='object')

Таблица с несколькими разными типами данных.


In [114]:
list(range(1,5))

[1, 2, 3, 4]

In [121]:
pd.Series(1, index=list('abcd'),
                                    dtype='float32')

a    1.0
b    1.0
c    1.0
d    1.0
dtype: float32

In [125]:
df2 = pd.DataFrame({ 'A': 1.,
                     'B': pd.Timestamp('20130102'),
                     'C': pd.Series(1, index=list(range(4)),
                                    dtype='int32'),
                     'D': np.array([3] * 4,
                                   dtype='int32'),
                     'E': pd.Categorical(["test", "train",
                                       "test", "train"]), # one - hot encoding
                     'F': 'foo'})
df2


,A,B,C,D,E,F
0,1.0,2013-01-02,1,3,test,foo
1,1.0,2013-01-02,1,3,train,foo
2,1.0,2013-01-02,1,3,test,foo
3,1.0,2013-01-02,1,3,train,foo


In [73]:
df2['B'].dt.year

0    2013
1    2013
2    2013
3    2013
Name: B, dtype: int32

In [74]:
df2['B'].dt.month

0    1
1    1
2    1
3    1
Name: B, dtype: int32

In [75]:
df2['B'].dt.day

0    2
1    2
2    2
3    2
Name: B, dtype: int32

In [76]:
df2['B']+pd.Timedelta(4,'w')

0   2013-01-30
1   2013-01-30
2   2013-01-30
3   2013-01-30
Name: B, dtype: datetime64[ns]

In [77]:
df2['B']

0   2013-01-02
1   2013-01-02
2   2013-01-02
3   2013-01-02
Name: B, dtype: datetime64[ns]

Анализ DataFrame.
--

In [78]:
df.head()  # в аргументе можем указать сколько первых строк хотим вывести
df.tail(3)  # (а здесь последних). по умолчанию – 5
df.index  # индексы
df.columns 
df.values

array([[ 0.        ,  0.        , -0.14653158],
       [ 1.        ,  1.        , -0.77040849],
       [        nan,  2.        , -0.47084668],
       [ 2.        ,  3.        , -0.98299741],
       [ 3.        ,  4.        , -1.06158791],
       [ 4.        ,  5.        ,  1.06839078],
       [ 5.        ,  6.        ,  0.23814966]])

При обращении к values мы получим знакомые numpy.array. Да, данные хранятся именно так.
Описательные статистики похожи на то что в Series, считаются по столбцам.

In [79]:
df.describe()


,one,two,three
count,6.000000,7.000000,7.000000
mean,2.500000,3.000000,-0.303690
std,1.870829,2.160247,0.761854
min,0.000000,0.000000,-1.061588
25%,1.250000,1.500000,-0.876703
50%,2.500000,3.000000,-0.470847
75%,3.750000,4.500000,0.045809
max,5.000000,6.000000,1.068391


Данные можно транспонировать – как бы поменять местами строки со столбцами


In [80]:
df.T


,a,b,c,d,e,f,g
one,0.000000,1.000000,NaN,2.000000,3.000000,4.000000,5.00000
two,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.00000
three,-0.146532,-0.770408,-0.470847,-0.982997,-1.061588,1.068391,0.23815


Сортировка по столбцу, например, по третьему


In [81]:
df.sort_values(by='three', ascending=False)


,one,two,three
f,4.0,5,1.068391
g,5.0,6,0.238150
a,0.0,0,-0.146532
c,NaN,2,-0.470847
b,1.0,1,-0.770408
d,2.0,3,-0.982997
e,3.0,4,-1.061588


Можно проитерироваться по столбцам


In [82]:
for i in df.columns:
  print(i, df[i])


one a    0.0
b    1.0
c    NaN
d    2.0
e    3.0
f    4.0
g    5.0
Name: one, dtype: float64
two a    0
b    1
c    2
d    3
e    4
f    5
g    6
Name: two, dtype: int64
three a   -0.146532
b   -0.770408
c   -0.470847
d   -0.982997
e   -1.061588
f    1.068391
g    0.238150
Name: three, dtype: float64


Или по строкам. И то, и то неэффективно и следует по возможности избегать прямой итерации, заменяя её операциями над датафреймами, о которых мы поговорим позже.


In [83]:
for i, j in df.iterrows():
  print(i,j)


a one      0.000000
two      0.000000
three   -0.146532
Name: a, dtype: float64
b one      1.000000
two      1.000000
three   -0.770408
Name: b, dtype: float64
c one           NaN
two      2.000000
three   -0.470847
Name: c, dtype: float64
d one      2.000000
two      3.000000
three   -0.982997
Name: d, dtype: float64
e one      3.000000
two      4.000000
three   -1.061588
Name: e, dtype: float64
f one      4.000000
two      5.000000
three    1.068391
Name: f, dtype: float64
g one      5.00000
two      6.00000
three    0.23815
Name: g, dtype: float64


*Задание для закрепления.
Сгенерируйте массив точек в 3D, создайте по нему датафрейм и отсортируйте строки в порядке следования осей.*


In [84]:
pd.DataFrame(
    np.random.normal(size=(100, 3)),
    columns=['x', 'y', 'z']
).sort_values(by=['x', 'y', 'z'])


,x,y,z
4,-2.904375,0.454166,-1.100485
53,-2.229346,2.411966,0.225889
37,-2.164886,-0.674176,-0.238169
65,-1.881564,-0.232201,-0.127590
82,-1.825766,0.563061,-0.901687
...,...,...,...
86,1.668996,-1.051459,-0.966243
23,1.828239,-1.513735,0.328563
49,1.858177,0.361421,-1.319767
31,2.258048,0.172585,-0.333104


Индексация в DataFrame.
--

В отличии от обычной системы индексации в Python и Numpy, в Pandas принята иная система индексации, которая является несколько нелогичной, однако, на практике часто оказывается удобной при обработке сильно неоднородных данных. Для написания продуктивного кода при обработке большого объема данных стоит использовать атрибуты .loc, .iloc.


In [85]:
df['one']  # Если в качестве индекса указать имя столбца, получится одномерный набор данных типа Series.
df.one  # К столбцу можно обращаться как к полю объекта, если имя столбца позволяет это сделать (= является строкой)
df['one'].index  # Индексы полученного одномерного набора данных.
df['one'].name  # Можно получить имя данного столбца
df['one']['d']  # Получение элемента массива. Сначала надо указать столбец, потом строку 
#df['c']['one']  # А наоборот нельзя!


2.0

Правила индексации в pandas несколько отличаются от общепринятых. Если указать диапазон индексов, то это означает диапазон строк. Причём последняя строка включается в таблицу.


In [86]:
df['b':'d']  # правая граница включена!
df[1:3]  # но диапазон целых чисел даёт диапазон строк с такими номерами, не включая правую границу (как обычно при индексировании списков). 
#df[1]  # при этом обратиться к индексу по номеру нельзя, а вот срез взять можно!
# Всё это кажется поначалу нелогичным и запутанным, хотя и удобно на практике (и на самом деле логично). 
df[['one', 'three']]
display(df['one'])
display(df[['one']])  # как вы думаете, чем обусловлено отличие в представлении этой и прошлой строки? 


a    0.0
b    1.0
c    NaN
d    2.0
e    3.0
f    4.0
g    5.0
Name: one, dtype: float64

,one
a,0.0
b,1.0
c,NaN
d,2.0
e,3.0
f,4.0
g,5.0


Логичнее работает атрибут loc: первая позиция — всегда индекс строки, а вторая — столбца. 


In [87]:
df.loc['b'] # .__index__
df.loc['b', 'one']
df['one']['b']  # эта строка выведет то же что и предыдущая 
df.loc['a':'b', 'one']  # можно использовать срезы по-всякому 
df.loc['a':'b', :]
df.loc[:, 'one']  # двоеточие, напомним, означает, что берем все значения по соответсвующей размерности
df.iloc[2]  # атрибут iloc подобен loc: первый индекс — номер строки, второй — номер столбца. Это целые числа, конец диапазона не включается как обычно в питоне 
df.iloc[1:3]
df.iloc[1:3, 0:2]  # и тут срезы абсолютно валидны


,one,two
b,1.0,1
c,NaN,2


In [129]:
df.loc['a','one':'two']

one    0.0
two    0.0
Name: a, dtype: float64

Булевская индексация — выбор строк с заданным условием.


 **Решение задач**

In [130]:
df[df.notna()]

,one,two,three
a,0.0,0,-0.146532
b,1.0,1,-0.770408
c,NaN,2,-0.470847
d,2.0,3,-0.982997
e,3.0,4,-1.061588
f,4.0,5,1.068391
g,5.0,6,0.238150


In [88]:
# создаем матрицу
n, m = 20, 10
data = np.random.randint(low=-100, high=100,size=(n, m))
cols = np.arange(1, m + 1)
np.random.shuffle(cols)

# создаем таблицу
task_df = pd.DataFrame(data, columns=cols)

# задаем условия для строк и столбцов
col_mask = (cols % 2) == 0
row_mask = np.sum(data % 2, axis=1) < (m / 2)

# извлекаем данные по условию
task_df.loc[row_mask, col_mask]


,4,2,10,8,6
1,90,44,-66,-97,-85
6,-12,65,96,-54,-63
7,-54,-70,94,69,-93
8,-94,-67,37,66,22
9,54,26,28,-95,98
10,-31,79,-63,16,-77
11,-40,-57,-76,-76,14
15,19,-60,61,-43,8
17,-30,62,-52,-8,-16
18,-78,-99,69,66,59
